# Preprocessing data

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df = pd.read_csv("raw_dataset/tracks.csv")

In [3]:
df

id                                 name  \
0       35iwgR4jXetI318WEWsa1Q                                Carve   
1       021ht4sdgPcrDgSk7JTbKY  Capítulo 2.16 - Banquero Anarquista   
2       07A5yehtSnoedViJAZkNnc   Vivo para Quererte - Remasterizado   
3       08FmqUhxtyLTn6pAh6bk45        El Prisionero - Remasterizado   
4       08y9GfoqCWfOGsKdwojr5e                  Lady of the Evening   
...                        ...                                  ...   
586667  5rgu12WBIHQtvej2MdHSH0                                  云与海   
586668  0NuWgxEp51CutD2pJoF4OM                                blind   
586669  27Y1N4Q4U3EfDU5Ubw8ws2            What They'll Say About Us   
586670  45XJsGpFTyzbzeWK8VzR8S                      A Day At A Time   
586671  5Ocn6dZ3BJFPWh4ylwFXtn                     Mar de Emociones   

        popularity  duration_ms  explicit                          artists  \
0                6       126903         0                          ['Uli']   
1                0        98200         0              ['Fernando Pessoa']   
2                0       181640         0              ['Ignacio Corsini']   
3                0       176907         0              ['Ignacio Corsini']   
4                0       163080         0                  ['Dick Haymes']   
...            ...          ...       ...                              ...   
586667          50       258267         0                      ['阿YueYue']   
586668          72       153293         0                   ['ROLE MODEL']   
586669          70       187601         0                      ['FINNEAS']   
586670          58       142003         0  ['Gentle Bones', 'Clara Benin']   
586671          38       214360         0                    ['Afrosound']   

                                               id_artists release_date  \
0                              ['45tIt06XoI0Iio4LBEVpls']   1922-02-22   
1                              ['14jtPCOoNZwquk5wd9DxrY']   1922-06-01   
2                              ['5LiOoJbxVSAMkBS2fUm3X2']   1922-03-21   
3                              ['5LiOoJbxVSAMkBS2fUm3X2']   1922-03-21   
4                              ['3BiJGZsyX9sJchTqcSA7Su']         1922   
...                                                   ...          ...   
586667                         ['1QLBXKM5GCpyQQSVMNZqrZ']   2020-09-26   
586668                         ['1dy5WNgIKQU6ezkpZs4y8z']   2020-10-21   
586669                         ['37M5pPGs6V1fchFJSgCguX']   2020-09-02   
586670  ['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...   2021-03-05   
586671                         ['0i4Qda0k4nf7jnNHmSNpYv']   2015-07-01   

        danceability  energy  key  loudness  mode  speechiness  acousticness  \
0              0.645  0.4450    0   -13.338     1       0.4510         0.674   
1              0.695  0.2630    0   -22.136     1       0.9570         0.797   
2              0.434  0.1770    1   -21.180     1       0.0512         0.994   
3              0.321  0.0946    7   -27.961     1       0.0504         0.995   
4              0.402  0.1580    3   -16.900     0       0.0390         0.989   
...              ...     ...  ...       ...   ...          ...           ...   
586667         0.560  0.5180    0    -7.471     0       0.0292         0.785   
586668         0.765  0.6630    0    -5.223     1       0.0652         0.141   
586669         0.535  0.3140    7   -12.823     0       0.0408         0.895   
586670         0.696  0.6150   10    -6.212     1       0.0345         0.206   
586671         0.686  0.7230    6    -7.067     1       0.0363         0.105   

        instrumentalness  liveness  valence    tempo  time_signature  
0               0.744000    0.1510   0.1270  104.851               3  
1               0.000000    0.1480   0.6550  102.009               1  
2               0.021800    0.2120   0.4570  130.418               5  
3               0.918000    0.1040   0.3970  169.980               3  
4               0.130000    0.311

In [5]:
new_df = df.drop(columns=["name","popularity","duration_ms","artists","id_artists","release_date"])
new_df["sample_mp3_url"]=""

# Use Spotify API to get 30 seconds sample audio URL

In [36]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

with open("credentials.json","r") as f:
    credentials = json.loads(f.read())

auth_manager = SpotifyClientCredentials(client_id=credentials["client_id"], client_secret=credentials["client_secret"])
sp = spotipy.Spotify(auth_manager=auth_manager)

result = sp.tracks(tracks=["35iwgR4jXetI318WEWsa1Q","021ht4sdgPcrDgSk7JTbKY"])

for track in result["tracks"]:
    print(track["preview_url"])

https://p.scdn.co/mp3-preview/2f41db376b64c4b148cf426ef9a86e16ebcc92d9?cid=4ffd8b3c6aac481ea6b01a48d4f6d478
https://p.scdn.co/mp3-preview/d9c4623d97c4e872eccefae6bfa97a46094cc481?cid=4ffd8b3c6aac481ea6b01a48d4f6d478


In [97]:
url_list = []
for batch_i in tqdm(range(len(new_df["id"])//50+1)):
    tracks = new_df["id"][batch_i*50:batch_i*50+50]
    result = sp.tracks(tracks=tracks)
    for i in range(len(result["tracks"])):
        new_df.loc[batch_i*50+i,"sample_mp3_url"]=result["tracks"][i]["preview_url"]
    new_df[batch_i*50:batch_i*50+50].to_csv("dataset/tracks_url.csv",mode='a', index=False, header=False)

  0%|          | 7/11734 [00:02<1:04:05,  3.05it/s]


KeyboardInterrupt: 

In [12]:
new_df=new_df[50:].reset_index(drop=True)

In [13]:
new_df

level_0   index                      id  explicit  danceability  \
0            50     150  3eMrYc092k7SIJfWJ7oasR         0         0.831   
1            51     151  2AZgaYZSwUosJD71J2N2Zo         0         0.537   
2            52     152  7lRFR5GJCxK87ZbVMtQSeS         0         0.665   
3            53     153  6XTvSCqGLLH0vzQQRn55hz         0         0.709   
4            54     154  6qRvnXftofjYJm1Mg98UWL         0         0.693   
...         ...     ...                     ...       ...           ...   
586517   586567  586667  5rgu12WBIHQtvej2MdHSH0         0         0.560   
586518   586568  586668  0NuWgxEp51CutD2pJoF4OM         0         0.765   
586519   586569  586669  27Y1N4Q4U3EfDU5Ubw8ws2         0         0.535   
586520   586570  586670  45XJsGpFTyzbzeWK8VzR8S         0         0.696   
586521   586571  586671  5Ocn6dZ3BJFPWh4ylwFXtn         0         0.686   

        energy  key  loudness  mode  speechiness  acousticness  \
0       0.2620    8   -12.386     1       0.0942         0.984   
1       0.0443    3   -16.474     0       0.0343         0.996   
2       0.2540    8    -9.879     1       0.1080         0.976   
3       0.0620    5   -12.207     1       0.0480         0.995   
4       0.0270    0   -13.506     0       0.0562         0.992   
...        ...  ...       ...   ...          ...           ...   
586517  0.5180    0    -7.471     0       0.0292         0.785   
586518  0.6630    0    -5.223     1       0.0652         0.141   
586519  0.3140    7   -12.823     0       0.0408         0.895   
586520  0.6150   10    -6.212     1       0.0345         0.206   
586521  0.7230    6    -7.067     1       0.0363         0.105   

        instrumentalness  liveness  valence    tempo  time_signature  \
0               0.912000    0.2040   0.9010  104.606               4   
1               0.000265    0.1520   0.1370   80.468               4   
2               0.002170    0.6440   0.5030  125.734               4   
3               0.003410    0.1200   0.5530   85.854               4   
4               0.000000    0.1340   0.4020   75.749               4   
...                  ...       ...      ...      ...             ...   
586517          0.000000    0.0648   0.2110  131.896               4   
586518          0.000297    0.0924   0.6860  150.091               4   
586519          0.000150    0.0874   0.0663  145.095               4   
586520          0.000003    0.3050   0.4380   90.029               4   
586521          0.000000    0.2640   0.9750  112.204               4   

       sample_mp3_url  
0                      
1                      
2                      
3                      
4                      
...               ...  
586517                 
586518                 
586519                 
586520                 
586521                 

[586522 rows x 17 columns]